## BAX-BCL2(GROUP)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

data_bax = pd.read_csv('RT-PCR (BAX).csv')
data_bcl_2 = pd.read_csv('RT-PCR (BCL_2).csv')


treatments_bax = data_bax['TREATMENT']
treatments_bcl_2 = data_bcl_2['TREATMENT']
fold_values_bax = data_bax[['FOLD CHANGE1', 'FOLD CHANGE2', 'FOLD CHANGE3']]
fold_values_bcl_2 = data_bcl_2[['FOLD CHANGE1', 'FOLD CHANGE2', 'FOLD CHANGE3']]


data_bax['Mean_FOLD_CHANGE'] = fold_values_bax.mean(axis=1)
data_bax['STD_FOLD_CHANGE'] = fold_values_bax.std(axis=1)

data_bcl_2['Mean_FOLD_CHANGE'] = fold_values_bcl_2.mean(axis=1)
data_bcl_2['STD_FOLD_CHANGE'] = fold_values_bcl_2.std(axis=1)


combined_treatments = treatments_bax.tolist()
combined_mean_fold_values_bax = data_bax['Mean_FOLD_CHANGE'].tolist()
combined_mean_fold_values_bcl_2 = data_bcl_2['Mean_FOLD_CHANGE'].tolist()
combined_std_fold_values_bax = data_bax['STD_FOLD_CHANGE'].tolist()
combined_std_fold_values_bcl_2 = data_bcl_2['STD_FOLD_CHANGE'].tolist()


bar_width = 0.25
index = np.arange(len(combined_treatments))


plt.figure(figsize=(12, 6))


plt.bar(index, combined_mean_fold_values_bax, bar_width, label='BAX', edgecolor = 'black', color='lightskyblue')
plt.bar(index + bar_width, combined_mean_fold_values_bcl_2, bar_width, label='BCL_2', edgecolor = 'black', color='mediumorchid')


for i in range(len(combined_treatments)):
    plt.errorbar(index[i], combined_mean_fold_values_bax[i], yerr=combined_std_fold_values_bax[i], fmt='o', color="black", capsize=5)

for i in range(len(combined_treatments)):
    plt.errorbar(index[i] + bar_width, combined_mean_fold_values_bcl_2[i], yerr=combined_std_fold_values_bcl_2[i], fmt='o', color="black", capsize=5)

plt.xticks(index + bar_width / 2, combined_treatments, rotation=45, ha="right")

plt.xlabel('Treatments')
plt.ylabel('Fold Change/GAPDH')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.savefig('OUTPUT1.tif', dpi=600, format='tif')
plt.show()


## BAX ANALYSIS

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

df = pd.read_csv("RT-PCR (BAX).csv")

df_melted = df.melt(id_vars='TREATMENT', var_name='Group', value_name='Values')

f_stat, p_val = f_oneway(*[group['Values'] for name, group in df_melted.groupby('TREATMENT')])

print("ANOVA Results using scipy:")
print("F-statistic:", f_stat)
print("p-value: {:.10f}".format(p_val)) 


tukey_result = pairwise_tukeyhsd(df_melted['Values'], df_melted['TREATMENT'])

print("\nTukey's HSD Test using statsmodels:")
print(tukey_result)


## BCL_2 ANALYSIS

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd


df = pd.read_csv("RT-PCR (BCL_2).csv")

df_melted = df.melt(id_vars='TREATMENT', var_name='Group', value_name='Values')


f_stat, p_val = f_oneway(*[group['Values'] for name, group in df_melted.groupby('TREATMENT')])

print("ANOVA Results using scipy:")
print("F-statistic:", f_stat)
print("p-value: {:.10f}".format(p_val)) 


tukey_result = pairwise_tukeyhsd(df_melted['Values'], df_melted['TREATMENT'])

print("\nTukey's HSD Test using statsmodels:")
print(tukey_result)


## BAX/BCL2 (BAR CHART & ANALYSIS)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.stats.multicomp import MultiComparison
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm


data_BAX = pd.read_csv('RT-PCR (BAX).csv')
data_BCL2 = pd.read_csv('RT-PCR (BCL_2).csv')


data_BAX[['FOLD CHANGE1', 'FOLD CHANGE2', 'FOLD CHANGE3']] /= data_BCL2[['FOLD CHANGE1', 'FOLD CHANGE2', 'FOLD CHANGE3']]
data_BAX['Mean_Fold_Change'] = data_BAX[['FOLD CHANGE1', 'FOLD CHANGE2', 'FOLD CHANGE3']].mean(axis=1)
data_BAX['SD_Fold_Change'] = data_BAX[['FOLD CHANGE1', 'FOLD CHANGE2', 'FOLD CHANGE3']].std(axis=1)

data_BAX_long = pd.melt(data_BAX, id_vars=['TREATMENT'], value_vars=['FOLD CHANGE1', 'FOLD CHANGE2', 'FOLD CHANGE3'], 
                        var_name='Replicate', value_name='Fold_Change')


formula = 'Fold_Change ~ C(TREATMENT)'
model = ols(formula, data=data_BAX_long).fit()
anova_table = anova_lm(model)


multi_comp = MultiComparison(data_BAX_long['Fold_Change'], data_BAX_long['TREATMENT'])
tukey_results = multi_comp.tukeyhsd()


print("ANOVA Results:")
print(anova_table)


print("\nTukey's HSD Test Results:")
print(tukey_results)


plt.figure(figsize=(10, 6))


treatments = data_BAX_long['TREATMENT'].unique()
mean_fold_change_values = data_BAX['Mean_Fold_Change']
sd_fold_change_values = data_BAX['SD_Fold_Change']


colors = ['skyblue', 'limegreen', 'lightcoral', 'orange', 'maroon','mediumspringgreen','khaki','orchid']
plt.bar(treatments, mean_fold_change_values, yerr=sd_fold_change_values, width=0.35 ,edgecolor = 'black', color=colors, capsize=5)
plt.xlabel('Treatments')
plt.ylabel('Fold Change (BAX/BCL2)')
plt.xticks(rotation=45, ha='right')
plt.grid(True)



plt.tight_layout()
plt.savefig('OUTPUT2.tif', dpi=600, format='tif')
plt.show()

